# Split Area Of Interest into tiles

---

This notebook does the following:

1. Define area
    * the boundary of the City of Cape Town (CoCT) was taken from the [Municipal Demarcation Board](http://www.demarcation.org.za/site/?page_id=5160) 
2. Convert it to selected CRS
    * Lets go with UTM (UTM_34S)
3. Tile it into non-overlaping BBOXes
    * use `UtmZoneSplitter` from `sentinelhub` package    
    * select a small area for classification
4. Visualise the splitting
5. Store the BBOXes to disk
    * pickled list 
    * geopandas dataframe
    
**These BBOXes will be filled with reference data and satelite imagery in the following notebooks.**

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image
import os
from sentinelhub import BBoxSplitter, BBox, CRS, DataSource, UtmZoneSplitter

In [3]:
from pathlib import Path

In [4]:
import geopandas as gpd
import pandas as pd

In [5]:
path = Path('data/')

In [ ]:
#set .crs
UTM34_crs = CRS.UTM_34S

In [ ]:
UTM34_crs

## Load Area-of-Interest (AOI) 

In [ ]:
city = gpd.read_file(str(path/'metropolitan_municipality_cpt.shp'))

In [ ]:
city.plot()

In [ ]:
#check .crs
city.crs

In [ ]:
#if no .crs defined... find out what it is.. and define one
city.crs = "EPSG:4326"

In [ ]:
city.crs

In [ ]:
#reproject
city = city.to_crs(crs=UTM34_crs.pyproj_crs())

In [ ]:
city.plot()

In [ ]:
#check
city.crs

In [ ]:
#buffer
city_buffer = city.buffer(500)

In [ ]:
#get shape
city_shape = city_buffer.geometry.tolist()[-1]

### Estimate the size of the area

In [ ]:
city_shape.bounds

In [ ]:
print('Dimension of the area is {0:.0f} x {1:.0f} m2'
      .format(city_shape.bounds[2]-city_shape.bounds[0],city_shape.bounds[3]-city_shape.bounds[1]))

In [ ]:
width_pix = int((city_shape.bounds[2]-city_shape.bounds[0])/10)
height_pix = int((city_shape.bounds[3]-city_shape.bounds[1])/10)

In [ ]:
print('Dimension of the area is {} x {} pixels: if pixels are 10m x 10m'.format(width_pix, height_pix))

# Perform tiling

The number of columns and tiles is selected in such a way that each EOPatch will have around 500 x 500 pixels at 10 meter resolution (10 sqkm).

In [ ]:
bbox_splitter = UtmZoneSplitter([city_shape], UTM34_crs, 5000) 

In [ ]:
print('Area bounding box: {}\n'.format(bbox_splitter.get_area_bbox().__repr__()))

In [ ]:
bbox_list = bbox_splitter.get_bbox_list()
info_list = bbox_splitter.get_info_list()

print('Each bounding box also has some info. on how it was created. Example:\n'
      '\nbbox: {}\ninfo: {}\n'.format(bbox_list[0].__repr__(), info_list[0]))

In [ ]:
print("The total number of tiles are",len(bbox_splitter.bbox_list))

###  Convert to geopandas dataframe

In [ ]:
#create a a gpd
from shapely.geometry import Polygon

In [ ]:
geometry = [Polygon(bbox.get_polygon()) for bbox in bbox_splitter.bbox_list]
idxs_x = [info['index_x'] for info in bbox_splitter.info_list]
idxs_y = [info['index_y'] for info in bbox_splitter.info_list]

df = pd.DataFrame({'index_x':idxs_x, 'index_y':idxs_y})
common_crs = bbox_splitter.bbox_list[0].crs
geodf = gpd.GeoDataFrame(df, crs=common_crs.pyproj_crs(), geometry=geometry)

In [ ]:
geodf.head()

In [ ]:
print(len(geodf))

In [ ]:
#save the shape to pickle
import pickle

if not os.path.isdir('./data/tile-def'):
    os.makedirs('./data/tile-def')

In [ ]:
# Within the area of interest select a smaller 5x5 area 

#id of center patch
ID = 86

# Obtain surrounding 5x5 patches
patchIDs = []
for idx, [bbox, info] in enumerate(zip(bbox_list, info_list)):
    if (abs(info['index_x'] - info_list[ID]['index_x']) <= 2 and
        abs(info['index_y'] - info_list[ID]['index_y']) <= 2):
        patchIDs.append(idx)

# Check if final size is 5x5
if len(patchIDs) != 5*5:
    print('Warning! Use a different central patch ID, this one is on the border.')
    
# Change the order of the patches (used for plotting later)
patchIDs = np.transpose(np.fliplr(np.array(patchIDs).reshape(5, 5))).ravel()

#save to pickle
#name_crs_columns_rows_numbertiles
with open('./data/tile-def/CoCT_500x500.pickle','wb') as fp:
    pickle.dump(bbox_splitter, fp)

In [ ]:
#save the smaller selection to .shp
smallerSelection = geodf[geodf.index.isin(patchIDs)]
smallerSelection.head()

In [ ]:
print(len(smallerSelection))

In [ ]:
# save the small selection to shapefile 
shapefile_name = './data/tile-def/smallerSelection_CoCT_500x500.shp'
smallerSelection.to_file(shapefile_name)

## Visualise

In [ ]:
fontdict = {'family': 'monospace', 'weight': 'normal', 'size': 11}

# if bboxes have all same size, estimate offset
xl, yl, xu, yu = geodf.geometry[0].bounds
xoff, yoff = (xu-xl)/3, (yu-yl)/5

# plot figure
fig, ax = plt.subplots(figsize=(24,18))
geodf.plot(ax=ax,facecolor='w',edgecolor='r',alpha=0.5)
#smallerSelection.plot(ax=ax, facecolor='g',edgecolor='r',alpha=0.4)
city.plot(ax=ax, facecolor='w',edgecolor='b',alpha=0.5)
city_buffer.plot(ax=ax, facecolor='w',edgecolor='g',alpha=0.5)

ax.set_title('City of Cape Town (with selection) in a 500 x 500 grid');
# add annotiation text
for idx in geodf.index:
    eop_name = '{0}x{1}'.format(geodf.index_x[idx], geodf.index_y[idx])
    centroid, = list(geodf.geometry[idx].centroid.coords)
    ax.text(centroid[0]-xoff, centroid[1]+yoff, '{}'.format(idx), fontdict=fontdict)
    ax.text(centroid[0]-xoff, centroid[1]-yoff, eop_name, fontdict=fontdict)

geodf[geodf.index.isin(patchIDs)].plot(ax=ax,facecolor='g',edgecolor='r',alpha=0.5)
    
save the figure
if not os.path.isdir('./figs'):
    #os.makedirs('./figs')
fig.savefig('./figs/aoi_500x500_tiles.png', bbox_inches='tight')

![title](./figs/aoi_500x500_tiles.png)

In [ ]:
#save the grid of the bigger area
geodf.to_file('./data/tile-def/city_bbox_32734_500x500_160.shp')